In [94]:
import numpy as np
import tensorflow as tf
import cv2


class BallDetector:
    """ Class contains functions for detecting a hand on an image.
        Args:   PATH_TO_GRAPH - path to neural network graph
                PATH_TO_LABELS - path to label map
                NUM_CLASSES - num classes in label map """

    def __init__(self, path_to_graph='PATH_TO_MODEL'):

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(path_to_graph, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            self.sess = tf.Session(graph=self.detection_graph)

        self.scores = self.detection_graph.get_tensor_by_name('detection_scores:0')

    def detect_balls(self, img_np, score_thresh=0.5):
        """ Function return a list with all founded hands.
            Args:
                image_np - image as numpy array. In openCV image is already numpy array
                score_thresh - min score of network's confidence
                             that founded object is hand (0: 1)"""

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(img_np, axis=0)
        image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

        # Actual detection.
        (boxes, scores, classes, num_detections) = self.sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        actual_boxes = []
        h, w, _ = img_np.shape
        for i in range(0, len(boxes[0])):
            if scores[0][i] > score_thresh:
                box = boxes[0][i]
                cv_box = []
                cv_box.append(int(box[1] * w))
                cv_box.append(int(box[0] * h))
                cv_box.append(int(box[3] * w))
                cv_box.append(int(box[2] * h))
                actual_boxes.append(cv_box)

        return actual_boxes

    def draw_box(self, img, box):
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), thickness=6)

    def stop(self):
        self.sess.close()

In [2]:
import os
from random import randint
import cv2


def get_random_img(images_dir):
    images_lst = os.listdir(images_dir)
    rand_img_name = images_lst[randint(0, len(images_lst)-1)]
    return cv2.imread(os.path.join(images_dir, rand_img_name))    

In [24]:
images_dir = "/home/ivan/Documents/Ball/data/ball"

rand_img = get_random_img(images_dir)

detector = BallDetector("PATH_TO_MODEL")
bboxes = detector.detect_balls(rand_img, score_thresh=0.7)

for b in bboxes:
    detector.draw_box(rand_img, b)

detector.stop()    

cv2.namedWindow("i", cv2.WINDOW_NORMAL)

cv2.imshow("i", rand_img)
cv2.waitKey()
cv2.destroyAllWindows()

[[0.9781835  0.94304097 0.941838   0.9417182  0.90160656 0.8866601
  0.8861246  0.8280619  0.8242534  0.6896413  0.66483164 0.65965235
  0.6579592  0.6013725  0.56414926 0.50984883 0.49257585 0.46618605
  0.38307393 0.3406133  0.3302     0.31919396 0.26889682 0.22356865
  0.20994884 0.20752421 0.18904933 0.16675597 0.16129035 0.15665689
  0.15282372 0.15177655 0.15015665 0.1467695  0.14450887 0.14398798
  0.13934585 0.12891698 0.12461421 0.12380663 0.12053165 0.11759478
  0.11360058 0.11291698 0.10856557 0.10781896 0.10766661 0.10679406
  0.10578266 0.10519913 0.105057   0.10468915 0.10459962 0.10310823
  0.10257596 0.1022076  0.10161337 0.09947157 0.09904784 0.09745827
  0.09717005 0.09482414 0.09463021 0.0944733  0.09427917 0.0937919
  0.09218091 0.09186447 0.09110498 0.09101003 0.09100598 0.09050295
  0.08972889 0.08858603 0.08823916 0.08805397 0.08750266 0.08723563
  0.08645245 0.08567068 0.0853596  0.08512115 0.08490193 0.0848968
  0.08361834 0.08361757 0.08338246 0.08199868 0.081